# FBI Uniform Crime Reporting - Quick Start Guide

This notebook demonstrates how to use the **FBIUCRConnector** to access FBI crime statistics.

**Data Categories:**
- **Violent Crime:** Murder, rape, robbery, aggravated assault
- **Property Crime:** Burglary, larceny-theft, motor vehicle theft, arson
- **Arrest Data:** Arrests by offense, demographics

**Geographic Levels:**
- National (aggregate)
- State
- Agency (individual police departments)

**API Access:** No API key required (public access)

---

*© 2025 KR-Labs. Licensed under Apache-2.0.*

## 1. Setup and Import

In [ ]:
from krl_data_connectors.crime import FBIUCRConnector
import pandas as pd

# Initialize connector
fbi = FBIUCRConnector()

print("✓ FBI UCR connector initialized")
print(f"API Base URL: {fbi.base_url}")

## 2. Get State Crime Data

Retrieve crime statistics for a state.

In [ ]:
# Get Rhode Island crime data for 2023
ri_crime = fbi.get_state_crime_data('RI', year=2023)

print(f"Retrieved crime data: {len(ri_crime)} records")
print("\nColumns:", ri_crime.columns.tolist())
ri_crime.head()

## 3. Load Crime Data from File

Alternative: Load downloaded UCR data from CSV.

In [ ]:
# If you have downloaded UCR data files
# Download from: https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/downloads

# crime_file = "path/to/ucr_crime_data.csv"
# crime_data = fbi.load_crime_data(crime_file)
# print(f"Loaded {len(crime_data)} crime records")

## 4. Extract Violent Crime

Filter for violent crime categories.

In [ ]:
# Get violent crime data
violent = fbi.get_violent_crime(ri_crime)

print("Violent Crime Categories:")
print(violent.columns.tolist())
print("\nViolent Crime Data:")
violent.head()

## 5. Extract Property Crime

Filter for property crime categories.

In [ ]:
# Get property crime data
property_crime = fbi.get_property_crime(ri_crime)

print("Property Crime Categories:")
print(property_crime.columns.tolist())
print("\nProperty Crime Data:")
property_crime.head()

## 6. Calculate Crime Rates

Calculate crime rates per 100,000 population.

In [ ]:
# Assuming crime data has population column
if 'population' in ri_crime.columns and 'violent_crime' in ri_crime.columns:
    # Calculate violent crime rate per 100,000
    with_rates = fbi.calculate_crime_rate(
        ri_crime,
        crime_col='violent_crime',
        population_col='population',
        per_capita=100000
    )
    
    print("Crime Rates (per 100,000 population):")
    print(with_rates[['state', 'violent_crime', 'population', 'violent_crime_rate']].head())
else:
    print("Sample crime rate calculation:")
    print("Violent crimes: 500")
    print("Population: 180,000")
    print(f"Rate per 100k: {(500/180000)*100000:.2f}")

## 7. Compare States

Compare crime statistics across multiple states.

In [ ]:
# Compare Northeast states
states = ['RI', 'MA', 'CT']
comparison = fbi.compare_states(states, year=2023, crime_type='violent')

print(f"State Comparison: {len(comparison)} records")
print("\nViolent Crime by State:")
comparison.head(10)

## 8. Year-over-Year Analysis

Calculate crime changes between years.

In [ ]:
# Get crime data for two consecutive years
crime_2023 = fbi.get_state_crime_data('RI', year=2023)
crime_2022 = fbi.get_state_crime_data('RI', year=2022)

# Calculate year-over-year change
if not crime_2023.empty and not crime_2022.empty and 'violent_crime' in crime_2023.columns:
    yoy_change = fbi.calculate_yoy_change(
        crime_2023,
        crime_2022,
        'violent_crime'
    )
    
    print("Year-over-Year Changes:")
    print(yoy_change[['state', 'violent_crime_current', 'violent_crime_previous',
                       'yoy_change', 'yoy_change_pct']].round(2))
else:
    print("Example YoY calculation:")
    print("2023: 500 violent crimes")
    print("2022: 450 violent crimes")
    print(f"Change: +50 (+11.1%)")

## 9. Multi-Year Trends

Analyze crime trends over multiple years.

In [ ]:
# Get 5-year trend data
trends = fbi.get_trend_data('RI', start_year=2019, end_year=2023)

if not trends.empty:
    print(f"Retrieved {len(trends)} years of trend data")
    print("\nCrime Trends (2019-2023):")
    if 'violent_crime' in trends.columns:
        print(trends[['year', 'violent_crime', 'property_crime']].sort_values('year'))
    else:
        print(trends.head())
else:
    print("No trend data available")

## 10. Crime Category Breakdown

Analyze specific crime types.

In [ ]:
# Show violent crime categories
print("Violent Crime Types:")
for crime in fbi.violent_crimes:
    print(f"  - {crime}")

print("\nProperty Crime Types:")
for crime in fbi.property_crimes:
    print(f"  - {crime}")

## 11. Export Results

Save crime data for further analysis.

In [ ]:
# Export state crime data
if not ri_crime.empty:
    fbi.export_to_csv(ri_crime, 'ri_crime_2023.csv')
    print("✓ Crime data exported")

# Export comparison
if not comparison.empty:
    fbi.export_to_csv(comparison, 'state_crime_comparison.csv')
    print("✓ Comparison exported")

## Next Steps

**Explore More:**
- Agency-level analysis (individual police departments)
- Arrest statistics by demographics
- Hate crime data
- Law enforcement officer statistics
- Clearance rates

**Resources:**
- [Crime Data Explorer](https://cde.ucr.cjis.gov/LATEST/webapp/)
- [FBI UCR Documentation](https://ucr.fbi.gov/)
- [API Documentation](https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi)

**Analysis Ideas:**
- Seasonal crime patterns
- Urban vs rural crime rates
- Crime hotspot identification
- Policy impact assessment
- Resource allocation optimization